In [10]:
import yaml
import pandas as pd

In [11]:
## Variable
config_file_path = "config.yml"

In [12]:
# functions
def config_loading(config_path:str):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

def date_dv_columns_check(data: pd.DataFrame, config: dict):
    required_columns = [config['date_column'], config['dv_column']] + config['data_prep_group_var']
    missing_columns = [col for col in required_columns if col not in data.columns]

    if missing_columns:
        raise ValueError(f"The following required columns are missing from the DataFrame: {missing_columns}")
    else:
        print("All required columns are present in the DataFrame.")

def data_date_conversion(data:pd.DataFrame, config:dict):
    data[config['date_column']] = pd.to_datetime(data[config['date_column']], format=config['date_format'])
    return data

def idv_list_loading(config:dict):
    idv = pd.read_csv(config['idv_list'])
    return idv

def check_idv_columns_in_data(data:pd.DataFrame, idv_list:pd.DataFrame, column_name:str):
    """
    Checks if all columns in df1 are present as rows in a specified column of df2.

    Parameters:
    - data (pd.DataFrame): The first DataFrame whose columns need to be checked.
    - idv_list (pd.DataFrame): The second DataFrame with the reference column.
    - column_name (str): The column in df2 that should contain all column names of df1.

    Returns:
    - None: If all columns are found, the function silently passes.

    Raises:
    - ValueError: If any columns are missing, it raises an error with the missing columns.
    """
    # Get the list of columns from df1
    df1_columns = set(data.columns)

    # Get the unique values in the specified column of df2
    df2_values = set(idv_list[column_name])

    # Find the missing columns
    missing_columns = df2_values - df1_columns

    # Raise an error if there are missing columns
    if missing_columns:
        raise ValueError(f"The following independent variables are missing in data: {missing_columns}")
    else:
        print("All independent variables in idv_list are present in the data.")

def data_loading(config:dict):

    input_data = pd.read_csv(config['input_data'])
    date_dv_columns_check(input_data, config)
    data = data_date_conversion(input_data, config)
    idv_list = idv_list_loading(config)
    check_idv_columns_in_data(data, idv_list, "idv_for_model_corrected")

    return data, idv_list

def filter_by_date_range(data:pd.DataFrame, config:dict):
    """Filtering date range for the data processing and further analysis

    Args:
        data (DataFrame): Harmonized_processed_data to filter out the date range
        config (dict): configuration dictionary
    Returns:
        DataFrame: Data with filtered date range
    """
    data["date"] = pd.to_datetime(data["date"], utc=False)

    # Print the minimum and maximum date values for verification
    print("Minimum date:", data["date"].min(skipna=True))
    print("Maximum date:", data["date"].max(skipna=True))

    # Define the date range from run_config
    date1 = pd.to_datetime(config['start_date'], format="%Y-%m-%d")
    date2 = pd.to_datetime(config['end_date'], format="%Y-%m-%d")

    # Filter the DataFrame based on the date range
    data = data[(data["date"] >= date1) & (data["date"] <= date2)]
    return data

### preprocessing functions ####

def cap_values(df, col, min_val, max_val):
    df[col] = df[col].astype(float).clip(lower=min_val, upper=max_val)
    return df


def get_unique_combinations(df: pd.DataFrame, columns: list):
    """
    Get unique combinations of values in the specified columns of a DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame to extract combinations from.
    - columns (list): A list of column names to consider.

    Returns:
    - list of dict: Each dictionary represents a unique combination of column-value pairs.
    """
    # Check if the specified columns exist in the DataFrame
    missing_columns = [col for col in columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"The following columns are missing from the DataFrame: {missing_columns}")

    # Extract unique rows for the specified columns
    unique_combinations = df[columns].drop_duplicates().to_dict(orient="records")

    return unique_combinations


def drop_rows_with_nulls(df: pd.DataFrame, group_by_columns: list, null_threshold: float):
    """
    Drop rows for specific combinations of columns where more than 50% of the 'value' column is null.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - group_by_columns (list): List of columns to group by.

    Returns:
    - pd.DataFrame: The DataFrame with rows removed based on the condition.
    """
    # Step 1: Group by specified columns
    grouped = df.groupby(group_by_columns)

    # Step 2: Calculate the percentage of nulls for each group
    null_percentage = grouped['value'].apply(lambda x: x.isnull().mean())

    # Step 3: Identify groups with more than 50% null values
    groups_to_drop = null_percentage[null_percentage > null_threshold].index

    # Step 4: Filter out rows belonging to the identified groups
    filtered_df = df[~df.set_index(group_by_columns).index.isin(groups_to_drop)]

    return filtered_df


def impute_groups(df: pd.DataFrame, group_by_columns: list, null_threshold: float, imputation_method: str = 'mean'):
    """
    Drop rows for specific combinations of columns where more than the specified percentage of the 'value' column is null,
    and impute missing values for groups with less than the threshold.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - group_by_columns (list): List of columns to group by.
    - null_threshold (float): The threshold of null percentage for which rows will be dropped.
    - imputation_method (str): The method for imputing missing values ('mean', 'median', 'mode').

    Returns:
    - pd.DataFrame: The DataFrame with rows removed and missing values imputed.
    """
    # Step 1: Group by specified columns
    grouped = df.groupby(group_by_columns)

    # Step 2: Calculate the percentage of nulls for each group
    null_percentage = grouped['value'].apply(lambda x: x.isnull().mean())

    # Step 3: Identify groups with more than the null threshold percentage of null values
    groups_to_drop = null_percentage[null_percentage > null_threshold].index

    # Step 4: Filter out rows belonging to the identified groups
    df_filtered = df[~df.set_index(group_by_columns).index.isin(groups_to_drop)]

    # Step 5: Impute missing values for the remaining groups (those with less than the threshold null percentage)
    for group, group_df in df_filtered.groupby(group_by_columns):
        if group not in groups_to_drop:
            if imputation_method == 'mean':
                fill_value = group_df['value'].mean()
            elif imputation_method == 'median':
                fill_value = group_df['value'].median()
            elif imputation_method == 'mode':
                fill_value = group_df['value'].mode()[0]
            else:
                raise ValueError(f"Unsupported imputation method: {imputation_method}")

            # Impute the missing values in the group
            df_filtered.loc[group_df.index, 'value'] = group_df['value'].fillna(fill_value)

    return df_filtered



In [13]:
config = config_loading(config_file_path)
data, idv_list = data_loading(config)
data = filter_by_date_range(data, config)

All required columns are present in the DataFrame.
All independent variables in idv_list are present in the data.
Minimum date: 2022-01-01 00:00:00
Maximum date: 2024-09-28 00:00:00


In [14]:
data

,brand,new_brand,category,date,directions_strategic_measures_brand_interest_index,directions_strategic_measures_brand_love_index,directions_strategic_measures_brand_loyalty_index,directions_brand_attributes_always_seems_to_be_doing_something_new_mean,directions_brand_attributes_creates_excitement_and_eagerness_at_mealtime_mean,directions_brand_attributes_has_a_taste_my_dog_enjoys_mean,...,directions_brand_attributes_has_treats_i_could_feed_my_dog_everyday_mean,directions_brand_attributes_helps_me_connect_with_my_dog_mean,directions_brand_attributes_is_designed_to_meet_my_dogs_health_needs_mean,directions_brand_attributes_offers_a_flavors_that_appeals_to_my_dogs_taste_mean,directions_brand_attributes_creates_playful_moments_with_my_cat_mean,directions_brand_attributes_has_craveable_treats_that_my_cat_comes_running_for_mean,directions_brand_attributes_has_treats_that_contain_wholesome_ingredients_mean,directions_brand_attributes_has_treats_that_have_an_appealing_package_mean,directions_brand_attributes_is_available_in_different_forms_for_different_occasions_mean,market_share
0,PEDIGREE,PEDIGREE,DOG FOOD,2022-01-01,0.306924,0.618816,0.595162,0.549426,0.663012,0.696708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063799
1,PEDIGREE,PEDIGREE,DOG FOOD,2022-01-01,0.306924,0.618816,0.595162,0.549426,0.663012,0.696708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064708
2,PEDIGREE,PEDIGREE,DOG FOOD,2022-01-01,0.306924,0.618816,0.595162,0.549426,0.663012,0.696708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063130
3,PEDIGREE,PEDIGREE,DOG FOOD,2022-01-01,0.306924,0.618816,0.595162,0.549426,0.663012,0.696708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065154
4,PEDIGREE,PEDIGREE,DOG FOOD,2022-01-01,0.306924,0.618816,0.595162,0.549426,0.663012,0.696708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,2024-06-01,0.820226,1.000000,0.885525,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.815158,0.849349,0.693033,NaN,0.768532,0.060428
4552,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,2024-06-01,0.820226,1.000000,0.885525,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.815158,0.849349,0.693033,NaN,0.768532,0.060560
4553,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,2024-06-01,0.820226,1.000000,0.885525,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.815158,0.849349,0.693033,NaN,0.768532,0.061395
4554,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,2024-06-01,0.820226,1.000000,0.885525,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.815158,0.849349,0.693033,NaN,0.768532,0.060086


In [15]:
melted_df = pd.melt(
    data,
    id_vars=['date', 'brand', 'new_brand', 'category'],  # Columns to keep as-is
    var_name='metric',  # Name of the new column for melted variable names
    value_name='value'  # Name of the new column for melted values
)


In [16]:
melted_df

,date,brand,new_brand,category,metric,value
0,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
1,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
2,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
3,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
4,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
...,...,...,...,...,...,...
265603,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,market_share,0.060428
265604,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,market_share,0.060560
265605,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,market_share,0.061395
265606,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,market_share,0.060086


In [17]:
impute_groups(melted_df, ['new_brand', 'category', 'metric'], .5)

,date,brand,new_brand,category,metric,value
0,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
1,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
2,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
3,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
4,2022-01-01,PEDIGREE,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.306924
...,...,...,...,...,...,...
261319,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,directions_brand_attributes_is_available_in_di...,0.768532
261320,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,directions_brand_attributes_is_available_in_di...,0.768532
261321,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,directions_brand_attributes_is_available_in_di...,0.768532
261322,2024-06-01,TEMPTATIONS,Stacked Brand,CAT TREATS ONLY,directions_brand_attributes_is_available_in_di...,0.768532


In [18]:
melted_df.shape

(265608, 6)